# Métricas del campeonato

## Posiciones

Este notebook describirá y calculará métricas de los campeonatos

Primero, se define $P_{k}^{if}$ como variable binaria que indica si el equipo $i$ en la fecha $f$ puede llegar a la posición $k$

Luego, definimos la métrica $F_{k}^f$ como la cantidad de equipos que pueden llegar a la posición $k$ en la fecha $f$.

Finalmente, se define la primera métrica $Champ_{k}$ como la cantidad de equipos que pueden llegar a la posición $k$.

La relación entre las variables es la siguiente:

$F_{k}^f = \sum_{i \in I} P_{k}^if$

$Champ_k = \sum_{f \in F} F_{k}^f$

Notese que para un campeonato perfecto, $Champ_k = |F||I|$

Luego, se define la métrica $ChampNorm_k = \frac{Champ_k}{|F||I|}$

## Rango

Sea $Pos_{i}^f$ La mejor posición que puede alcanzar el equipo $i$ en la fecha $f$ y $\hat{Pos_{i}}^f$ la peor, se define el rango como $Rango_i^f = \hat{Pos_{i}}^f - Pos_{i}^f$

Luego, se define el rango promedio de una fecha  como $RangFecha_f = \frac{\sum_{i \in I} Rango_i^f}{|I|}$

Por último, se define el rango promedio del campeonato como $RangChamp = \frac{\sum_{f \in f} RangFecha_f}{|F|}$

**TO DO:** Más metricas como cantidad de rangos máximos en un campeonato

## Cálculo

In [167]:
base_dir = '../logs/solutions'
champ = 'chile'
start_date = 24
filename_init = f'model-{champ}-init-0'
filename_opt = f'model-{champ}_20-opt-1'

In [174]:
beta_m, beta_p, z = {}, {}, {}


with open(f'{base_dir}/{champ}/{filename_opt}.sol', 'r', encoding='utf-8') as infile:
    for line in infile:
        line = line.strip()
        if 'beta' in line:
            var, value = line.split()
            _, var = var.split('[')
            var = var.strip(']')
            team, date = var.split(',')
            if 'beta_p' in line:
                beta_p[team, int(date)] = round(float(value))
            else:
                beta_m[team, int(date)] = round(float(value))
        if 'z' == line[0]:
            var, value = line.split()
            _, var = var.split('[')
            var = var.strip(']')
            team, date, p1, p2 = var.split(',')
            if float(value) > 0.5:
                value = 1
            else:
                value = 0
            z[(team, date, p1, p2)] = value

In [175]:
teams = list(set([i[0] for i in beta_m.keys()]))
dates = list(set([i[1] for i in beta_m.keys()]))

In [176]:
def can_team_date_achieve_position(team, date, position=1, best_case=True):
  if best_case:
    if beta_m[team, date] <= position:
      return True
    return False
  else:
    if beta_p[team, date] >= position:
      return True
    return False

In [177]:
def date_positions(date, position=1, best_case=True):
  tot = 0
  for team in teams:
    if can_team_date_achieve_position(team, date, position, best_case):
      tot += 1
  return tot

In [178]:
def champ_positions(position=1, best_case=True, norm=False):
  tot = 0
  for date in dates:
    tot += date_positions(date, position, best_case)
  if norm:
    tot /= (len(dates) * len(teams))
  return tot

In [179]:
print(champ_positions())
print(champ_positions(norm=True))

156
0.65


Rango

In [180]:
def team_date_range(team, date):
  return beta_p[team, date] - beta_m[team, date]

In [181]:
def date_range(date):
  total = 0
  for team in teams:
    total += team_date_range(team, date)
  return total / len(team)

In [182]:
def champ_range():
  total = 0
  for date in dates:
    total += date_range(date)
  return total / len(dates)

In [183]:
champ_range()

93.06666666666666